In [418]:
import os

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re

In [419]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [420]:
# Ratio de commandes par rapport au nombre de devis
ratio = .20

# Nombre de devis minimum pour être compté
min_devis = 5

# Nombre de commandes maximum pour être compté
max_commandes = 10

In [421]:
path_data = "../../data"

filename_entete = "entete.csv"

# équivalent à path_data + filename_meteo et s'assure que le path est de la bonne forme
path_data_entete = os.path.join(path_data, filename_entete)
# affiche le chemin path_data_meteo
print(path_data_entete)

../../data/entete.csv


In [422]:
df_entete = pd.read_csv(path_data_entete, sep=",")

In [423]:
df_entete

,CdeNumCom,TypCde,CodeClient,DateCreation,Gestionnaire,CAHT,Transfo
0,140500651,C,2625,4/10/2018,ALU,2065.80,140500651-02
1,140600145,C,5505,4/18/2018,ALU,16960.51,140600145-03
2,140600145-03,D,5505,4/17/2018,ALU,16967.47,140600145
3,140901111-01,D,3796,8/1/2018,PVC,1300.41,NaN
4,150200732,C,10497,7/30/2018,PVC,1748.40,150200732-02
...,...,...,...,...,...,...,...
73992,220400749-00,D,5191,4/19/2022,ALU,0.00,NaN
73993,220400750-00,D,14050,4/19/2022,BOIS,0.00,NaN
73994,220400751,C,1378,4/19/2022,ALU,0.00,NaN
73995,220400752,C,2048,4/19/2022,ALU,0.00,NaN


In [424]:
df_entete_devis = df_entete.loc[
    (df_entete["TypCde"] == "D") & (df_entete["CAHT"] > 0)
][{"CdeNumCom", "CodeClient", "Transfo"}]

df_entete_devis["Transfo"] = df_entete_devis["Transfo"].fillna(0)

df_entete_devis

,CdeNumCom,CodeClient,Transfo
2,140600145-03,5505,140600145
3,140901111-01,3796,0
5,150200732-02,10497,150200732
6,150201297-01,9746,0
7,150300298-01,4094,0
...,...,...,...
73946,220400697-00,2016,0
73954,220400705-00,14763,0
73981,220400733-00,4865,0
73983,220400735-00,1185,0


In [425]:
df_entete_devis["Nombre"] = 0

df_entete_devis["CdeNumCom"] = df_entete_devis["CdeNumCom"].str.extract('^(\d+?)-', expand=False)

df_entete_devis["Transfo"] = df_entete_devis["Transfo"] != 0

df_entete_devis = df_entete_devis.groupby(["CdeNumCom", "CodeClient", "Transfo"])["Nombre"].count().reset_index()

df_entete_devis["Nombre"] = 1

df_entete_devis

,CdeNumCom,CodeClient,Transfo,Nombre
0,140600145,5505,True,1
1,140901111,3796,False,1
2,150200732,10497,True,1
3,150201297,9746,False,1
4,150300298,4094,False,1
...,...,...,...,...
46264,220400697,2016,False,1
46265,220400705,14763,False,1
46266,220400733,4865,False,1
46267,220400735,1185,False,1


In [426]:
df_entete_transfo = pd.pivot_table(
    df_entete_devis,
    index=['CodeClient'],
    columns='Transfo',
    values='Nombre',
    aggfunc='count',
).reset_index().rename_axis(None, axis=1)

df_entete_transfo[True] = df_entete_transfo[True].fillna(0)
df_entete_transfo[False] = df_entete_transfo[False].fillna(0)

df_entete_transfo = df_entete_transfo.astype({
    "CodeClient": int,
    False: int,
    True: int
})

df_entete_transfo

,CodeClient,False,True
0,1,15,9
1,3,12,12
2,5,1,0
3,12,1,0
4,13,0,1
...,...,...,...
4243,14760,2,0
4244,14761,1,0
4245,14762,1,0
4246,14763,1,0


In [427]:
df_entete_transfo["Ratio"] = df_entete_transfo[True] / df_entete_transfo[False]

df_entete_transfo.replace([np.inf, -np.inf], 1, inplace=True)

df_entete_transfo

,CodeClient,False,True,Ratio
0,1,15,9,0.6
1,3,12,12,1.0
2,5,1,0,0.0
3,12,1,0,0.0
4,13,0,1,1.0
...,...,...,...,...
4243,14760,2,0,0.0
4244,14761,1,0,0.0
4245,14762,1,0,0.0
4246,14763,1,0,0.0


In [428]:
df_entete_transfo.loc[
    df_entete_transfo["Ratio"] < ratio
]

,CodeClient,False,True,Ratio
2,5,1,0,0.000000
3,12,1,0,0.000000
5,16,3,0,0.000000
6,22,7,1,0.142857
9,33,44,10,0.227273
...,...,...,...,...
4243,14760,2,0,0.000000
4244,14761,1,0,0.000000
4245,14762,1,0,0.000000
4246,14763,1,0,0.000000


In [429]:
df_entete

,CdeNumCom,TypCde,CodeClient,DateCreation,Gestionnaire,CAHT,Transfo
0,140500651,C,2625,4/10/2018,ALU,2065.80,140500651-02
1,140600145,C,5505,4/18/2018,ALU,16960.51,140600145-03
2,140600145-03,D,5505,4/17/2018,ALU,16967.47,140600145
3,140901111-01,D,3796,8/1/2018,PVC,1300.41,NaN
4,150200732,C,10497,7/30/2018,PVC,1748.40,150200732-02
...,...,...,...,...,...,...,...
73992,220400749-00,D,5191,4/19/2022,ALU,0.00,NaN
73993,220400750-00,D,14050,4/19/2022,BOIS,0.00,NaN
73994,220400751,C,1378,4/19/2022,ALU,0.00,NaN
73995,220400752,C,2048,4/19/2022,ALU,0.00,NaN


In [430]:
df_entete_commandessansdevis = df_entete.loc[
    (df_entete["TypCde"] == "C") & (df_entete["Transfo"].isna()) & (df_entete["CAHT"] > 0)
][{"CdeNumCom", "CodeClient"}]

df_entete_commandessansdevis["Nombre"] = 0

df_entete_commandessansdevis

,CdeNumCom,CodeClient,Nombre
636,180301212,645,0
637,180301213,10048,0
638,180301214,10048,0
640,180301216,10048,0
641,180301217,10048,0
...,...,...,...
73958,220400709,2334,0
73973,220400724,219,0
73976,220400727,10510,0
73977,220400728,9977,0


In [431]:
df_entete_commandessansdevis = df_entete_commandessansdevis.groupby(["CodeClient"])["Nombre"].count().reset_index()

df_entete_commandessansdevis

,CodeClient,Nombre
0,3,3
1,12,1
2,24,1
3,27,5
4,33,6
...,...,...
1727,14675,1
1728,14683,1
1729,14684,1
1730,14709,1


In [432]:
df_entete_transfo = pd.concat([df_entete_transfo, df_entete_commandessansdevis]).groupby("CodeClient").sum().reset_index()

df_entete_transfo

,CodeClient,False,True,Ratio,Nombre
0,1,15.0,9.0,0.6,0.0
1,3,12.0,12.0,1.0,3.0
2,5,1.0,0.0,0.0,0.0
3,12,1.0,0.0,0.0,1.0
4,13,0.0,1.0,1.0,0.0
...,...,...,...,...,...
4499,14761,1.0,0.0,0.0,0.0
4500,14762,1.0,0.0,0.0,0.0
4501,14763,1.0,0.0,0.0,0.0
4502,14764,0.0,0.0,0.0,1.0


In [433]:
df_entete_transfo[True] = df_entete_transfo[True] + df_entete_transfo["Nombre"]

df_entete_transfo["Ratio"] = df_entete_transfo[True] / df_entete_transfo[False]
df_entete_transfo.replace([np.inf, -np.inf], 1, inplace=True)

df_entete_transfo = df_entete_transfo.astype({
    "CodeClient": int,
    False: int,
    True: int,
    "Ratio": float,
    "Nombre": int
})

pd.set_option('display.max_rows', None)

df_entete_transfo

,CodeClient,False,True,Ratio,Nombre
0,1,15,9,0.600000,0
1,3,12,15,1.250000,3
2,5,1,0,0.000000,0
3,12,1,1,1.000000,1
4,13,0,1,1.000000,0
5,16,3,0,0.000000,0
6,22,7,1,0.142857,0
7,24,6,3,0.500000,1
8,27,7,10,1.428571,5
9,33,44,16,0.363636,6


In [456]:
df_entete_transfo.loc[
    (df_entete_transfo[False] >= 5) & (df_entete_transfo["Ratio"] >= ratio)
]

,CodeClient,False,True,Ratio,Nombre
0,1,15,9,0.600000,0
1,3,12,15,1.250000,3
7,24,6,3,0.500000,1
8,27,7,10,1.428571,5
9,33,44,16,0.363636,6
12,39,29,36,1.241379,24
13,43,76,45,0.592105,32
14,44,24,27,1.125000,18
15,45,12,4,0.333333,2
18,53,47,19,0.404255,12


In [455]:
df_entete_transfo.loc[
    (df_entete_transfo[False] >= 5) & (df_entete_transfo[True] >= 1) & (df_entete_transfo["Ratio"] < ratio)
]

,CodeClient,False,True,Ratio,Nombre
6,22,7,1,0.142857,0
20,62,31,7,0.225806,2
21,63,21,2,0.095238,1
45,122,39,8,0.205128,1
46,123,86,15,0.174419,0
49,135,25,4,0.160000,1
56,148,11,2,0.181818,0
57,149,14,2,0.142857,2
65,158,9,2,0.222222,1
81,216,5,1,0.200000,0


In [453]:
df_entete_transfo.loc[
    (df_entete_transfo[False] >= 5) & (df_entete_transfo[True] == 0)
]

,CodeClient,False,True,Ratio,Nombre
58,150,9,0,0.0,0
59,151,6,0,0.0,0
134,381,5,0,0.0,0
207,656,6,0,0.0,0
373,1254,8,0,0.0,0
452,1450,6,0,0.0,0
467,1489,6,0,0.0,0
468,1491,5,0,0.0,0
525,1638,5,0,0.0,0
528,1650,5,0,0.0,0
